# Lab 5

In [66]:
import matplotlib.pyplot as plt

import seaborn as sns
import torch 
from torch import nn

* Write ResNet architecture by pytorch.

I will try implement [ResNet-34](https://arxiv.org/pdf/1512.03385).

1. Implement resnet block

In [67]:
class ResBlock_34(nn.Module):
    """one block of resnet_34 model

    Args:
        in_out_channels (int): input and output channels
    """
    def __init__(self, in_out_channels):
        super().__init__()
        layers = list()  
        for i in range(2):
            layers.append(nn.Conv2d(in_out_channels, in_out_channels, kernel_size=1, stride=2))
            layers.append(nn.BatchNorm2d(in_out_channels))
            layers.append(nn.ReLU())
        self.layers = nn.Sequential(*layers)
        
    def forward(self, x):
        return self.layers(x) + x

2. Implement resnet layers

In [68]:
class conv2_x(nn.Module):
    def __init__(self, in_out_channels):
        """conv2_x layer

        Args:
            in_out_channels (int): input and output channels
        """
        super().__init__()
        layers = list()  
        layers.append(nn.MaxPool2d(3, 2))
        for i in range(3):
            layers.append(ResBlock_34(in_out_channels))

        
        self.layers = nn.Sequential(*layers)
        
    def forward(self, x):
        return self.layers(x)

In [69]:
class conv3_5_x(nn.Module):
    def __init__(self, in_out_channels, blocks):
        """conv3_x, conv4_x, conv5_x layers in one class

        Args:
            in_out_channels (int): input and output channels
            blocks (int): num of blocks
        """
        super().__init__()
        layers = list()  
        
        for i in range(blocks):           
            layers.append(ResBlock_34(in_out_channels))
            
        self.layers = nn.Sequential(*layers)
        
    def forward(self, x):
        return self.layers(x)

3. Implement resnet model

* implementation FC

* implementation resnet

In [74]:
class ResNet_34(nn.Module):
    def __init__(self, output_channels):
        super().__init__()
        # Convolutions
        blocks = [(128, 3), (256, 4), (256, 6), (512, 3)] # in_out_channels and num blocks
        layers = list()
        
        layers.append(nn.Conv2d(64, 64, kernel_size=7, stride=2))
        layers.append(conv2_x(64))
        for block in blocks:
            layers.append(conv3_5_x(block[0], block[1]))
        layers.append(nn.AvgPool2d(1))    
        
        # FC
        layers.append(nn.Linear(blocks[-1][0], output_channels))
        self.layers = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.layers(x)

In [75]:
ResNet_34(1)

ResNet_34(
  (layers): Sequential(
    (0): Conv2d(64, 64, kernel_size=(7, 7), stride=(2, 2))
    (1): conv2_x(
      (layers): Sequential(
        (0): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
        (1): ResBlock_34(
          (layers): Sequential(
            (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(2, 2))
            (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU()
            (3): Conv2d(64, 64, kernel_size=(1, 1), stride=(2, 2))
            (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (5): ReLU()
          )
        )
        (2): ResBlock_34(
          (layers): Sequential(
            (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(2, 2))
            (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU()
            (3): Conv2d(64, 64, kernel_size=(1, 1), stride=(2, 2))
   